In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from kcsd import KCSD1D
from scipy.signal import find_peaks
from scipy.ndimage import gaussian_filter1d
from pathlib import Path

In [ ]:
from ecephys.sglx_utils import load_timeseries
from ecephys_analyses.data import channel_groups, paths
from ecephys.plot import lfp_explorer, colormesh_timeseries_explorer
from ecephys.signal.utils import mean_subtract, median_subtract
from ecephys.signal.csd import get_kcsd

In [ ]:
def get_csd(sig):
    gdx = intersite_distance = 0.020
    k = get_kcsd(
        sig, intersite_distance=intersite_distance, gdx=gdx, do_lcurve=True
    )
    est_csd = k.values('CSD')
    
    return k.estm_x, est_csd

In [41]:
subject = "Valentino"
#chans = channel_groups.full[subject]
chans = channel_groups.drift_tracking[subject]
channel_indices = np.arange(len(chans))
start_time=0
end_time=30

In [42]:
N2_bin_path = Path('/Volumes/neuropixel/Data/CNPIX3-Valentino/state_samples/N2.exported.imec0.lf.bin')
N2_sig = load_timeseries(N2_bin_path, chans, start_time=start_time, end_time=end_time)
N2_pos, N2_csd = get_csd(N2_sig)

nChan: 385, nFileSamp: 150000
Performing L-curve parameter estimation...
No lambda given, using defaults
min lambda 1e-14
max lambda 0.0106
min lambda 1e-14
max lambda 0.0106
l-curve (all lambda):  0.23
Best lambda and R =  0.0006359501216457479 ,  0.23


In [43]:
N1_bin_path = Path('/Volumes/neuropixel/Data/CNPIX3-Valentino/state_samples/N1.exported.imec0.lf.bin')
N1_sig = load_timeseries(N1_bin_path, chans, start_time=start_time, end_time=end_time)
N1_pos, N1_csd = get_csd(N1_sig)

nChan: 385, nFileSamp: 150000
Performing L-curve parameter estimation...
No lambda given, using defaults
min lambda 1e-14
max lambda 0.0106
min lambda 1e-14
max lambda 0.0106
l-curve (all lambda):  0.23
Best lambda and R =  0.0006359501216457479 ,  0.23


In [44]:
QWK_bin_path = Path('/Volumes/neuropixel/Data/CNPIX3-Valentino/state_samples/QWK.exported.imec0.lf.bin')
QWK_sig = load_timeseries(QWK_bin_path, chans, start_time=start_time, end_time=end_time)
QWK_pos, QWK_csd = get_csd(QWK_sig)

nChan: 385, nFileSamp: 150000
Performing L-curve parameter estimation...
No lambda given, using defaults
min lambda 1e-14
max lambda 0.0106
min lambda 1e-14
max lambda 0.0106
l-curve (all lambda):  0.23
Best lambda and R =  0.0025957755090429014 ,  0.23


In [45]:
AWK_bin_path = Path('/Volumes/neuropixel/Data/CNPIX3-Valentino/state_samples/AWK.exported.imec0.lf.bin')
AWK_sig = load_timeseries(AWK_bin_path, chans, start_time=start_time, end_time=end_time)
AWK_pos, AWK_csd = get_csd(AWK_sig)

nChan: 385, nFileSamp: 150000
Performing L-curve parameter estimation...
No lambda given, using defaults
min lambda 1e-14
max lambda 0.0106
min lambda 1e-14
max lambda 0.0106
l-curve (all lambda):  0.23
Best lambda and R =  0.0006359501216457479 ,  0.23


In [46]:
REM_bin_path = Path('/Volumes/neuropixel/Data/CNPIX3-Valentino/state_samples/REM.exported.imec0.lf.bin')
REM_sig = load_timeseries(REM_bin_path, chans, start_time=start_time, end_time=end_time)
REM_pos, REM_csd = get_csd(REM_sig)

nChan: 385, nFileSamp: 150000
Performing L-curve parameter estimation...
No lambda given, using defaults
min lambda 1e-14
max lambda 0.0106
min lambda 1e-14
max lambda 0.0106
l-curve (all lambda):  0.23
Best lambda and R =  0.0025957755090429014 ,  0.23


In [47]:
#colormesh_timeseries_explorer(sig.time.values, est_csd.T, y=k.estm_x)

In [48]:
#165-170 should be SR?

In [49]:
def get_biggest_sinks(est_csd, smooth=False):
    biggest_sinks_by_channel = est_csd.min(axis=1)
    if smooth:
        biggest_sinks_by_channel = gaussian_filter1d(biggest_sinks_by_channel, sigma=0.5)
    peaks, properties = find_peaks(-biggest_sinks_by_channel, prominence=1000, distance=10)
    print(chans[peaks])

    fig, ax = plt.subplots(figsize=(25, 5))
    sns.barplot(x=channel_indices, y=biggest_sinks_by_channel, color='steelblue', ax=ax)
    ax.set(xlabel="Channel", ylabel="Biggest Sink Amplitude")
    ax.set_xticks(channel_indices[::2]);
    ax.set_xticklabels(chans[::2], rotation=90, fontsize=6);

    sns.scatterplot(x=peaks, y=biggest_sinks_by_channel[peaks], marker='x', color='red', ax=ax)

In [50]:
get_biggest_sinks(N2_csd)

[368  41 106 141 166 193 225]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [51]:
get_biggest_sinks(N2_csd, smooth=True)

[368  41 106 141 166 193 225]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [52]:
get_biggest_sinks(N1_csd)

[367  21  41  85 146 169 198 229]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [53]:
get_biggest_sinks(QWK_csd)

[ 46 101 142 169 197 241]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [54]:
get_biggest_sinks(AWK_csd)

[ 21  42 138 170 197]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [55]:
get_biggest_sinks(AWK_csd, smooth=True)

[ 21  42 138 170 197]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [56]:
get_biggest_sinks(REM_csd)

[ 34 105 146 197]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [57]:
def get_sink_fingerprint(est_csd):    
    nbins = 50
    bin_edges = np.linspace(0, -est_csd.min(), nbins)

    res = np.apply_along_axis(lambda x: np.histogram(x, bins=bin_edges), axis=1, arr=-est_csd)
    hist = np.vstack(res[:, 0])
    
    fig, ax = plt.subplots(figsize=(25, 5))
    sns.heatmap(np.log1p(hist.T), cbar=False)
    ax.set_xticks(channel_indices[::2]);
    ax.set_xticklabels(chans[::2], rotation=90, fontsize=6);
    ax.set_yticks([])
    ax.set(xlabel='Channel', ylabel='CSD amplitude count');

In [58]:
get_sink_fingerprint(N2_csd)

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [59]:
get_sink_fingerprint(N1_csd)

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [60]:
get_sink_fingerprint(N1_csd)

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [61]:
get_sink_fingerprint(QWK_csd)

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [62]:
get_sink_fingerprint(AWK_csd)

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:
get_sink_fingerprint(REM_csd)

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [64]:
colormesh_timeseries_explorer(REM_sig.time.values, REM_csd.T, y=REM_pos)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

In [65]:
def get_mean_sinks(est_csd, smooth=False):
    est_csd[est_csd > 0] = 0
    mean_sinks_by_channel = est_csd.mean(axis=1)
    if smooth:
        mean_sinks_by_channel = gaussian_filter1d(biggest_sinks_by_channel, sigma=0.5)
    peaks, properties = find_peaks(-mean_sinks_by_channel, prominence=1000, distance=10)
    print(chans[peaks])

    fig, ax = plt.subplots(figsize=(25, 5))
    sns.barplot(x=channel_indices, y=mean_sinks_by_channel, color='steelblue', ax=ax)
    ax.set(xlabel="Channel", ylabel="Mean Sink Amplitude")
    ax.set_xticks(channel_indices[::2]);
    ax.set_xticklabels(chans[::2], rotation=90, fontsize=6);

    sns.scatterplot(x=peaks, y=mean_sinks_by_channel[peaks], marker='x', color='red', ax=ax)

In [66]:
get_mean_sinks(N2_csd)

[141 173 237]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [67]:
get_mean_sinks(N1_csd)

[ 49 142 173 237]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [68]:
get_mean_sinks(QWK_csd)

[ 89 142]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [69]:
get_mean_sinks(AWK_csd)

[142 238]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [70]:
get_mean_sinks(REM_csd)

[ 93 146]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [71]:
fig, ax = plt.subplots()
sns.heatmap(np.cov(N2_csd), xticklabels=chans, yticklabels=chans, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [72]:
fig, ax = plt.subplots()
sns.heatmap(np.cov(N1_csd), xticklabels=chans, yticklabels=chans, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [73]:
fig, ax = plt.subplots()
sns.heatmap(np.cov(QWK_csd), xticklabels=chans, yticklabels=chans, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [74]:
fig, ax = plt.subplots()
sns.heatmap(np.cov(AWK_csd), xticklabels=chans, yticklabels=chans, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [75]:
fig, ax = plt.subplots()
sns.heatmap(np.cov(REM_csd), xticklabels=chans, yticklabels=chans, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [76]:
def plot_csd_variance(csd):
    fig, ax = plt.subplots(figsize=(25, 5))
    sns.barplot(x=channel_indices, y=np.diag(np.cov(csd)), color='steelblue', ax=ax)
    ax.set(xlabel="Channel", ylabel="CSD variance")
    ax.set_xticks(channel_indices[::2]);
    ax.set_xticklabels(chans[::2], rotation=90, fontsize=6);

In [77]:
plot_csd_variance(N2_csd)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [78]:
plot_csd_variance(N1_csd)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [79]:
plot_csd_variance(QWK_csd)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [80]:
plot_csd_variance(AWK_csd)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [81]:
plot_csd_variance(REM_csd)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …